In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('always')
from sklearn.model_selection import train_test_split
import shapely.speedups
from sklearn.preprocessing import StandardScaler
from IPython.display import Image
import re
from itertools import combinations
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

np.random.seed(1)
tf.random.set_seed(1)

In [2]:
viviendas_train=pd.read_csv('./TP1/Datasets/viviendas_caba_train.csv').copy()
viviendas_test=pd.read_csv('./TP1/Datasets/viviendas_caba_test.csv').copy()

viviendas_train=pd.read_csv('./TP1/Datasets/viviendas_caba_train.csv').copy()
viviendas_test=pd.read_csv('./TP1/Datasets/viviendas_caba_test.csv').copy()
viviendas = pd.concat([viviendas_train, viviendas_test])

In [4]:
viviendas = pd.get_dummies(viviendas, columns = ["property_type"]) 
viviendas = pd.get_dummies(viviendas, columns = ["barrio"])

In [6]:
viviendas.columns

Index(['start_date', 'end_date', 'latitud', 'longitud', 'property_rooms',
       'property_bedrooms', 'property_surface_total',
       'property_surface_covered', 'property_price', 'property_title',
       'geometry', 'property_type_Casa', 'property_type_Departamento',
       'property_type_PH', 'barrio_Abasto', 'barrio_Agronomía',
       'barrio_Almagro', 'barrio_Balvanera', 'barrio_Barracas',
       'barrio_Barrio Norte', 'barrio_Belgrano', 'barrio_Boca', 'barrio_Boedo',
       'barrio_Caballito', 'barrio_Catalinas', 'barrio_Centro / Microcentro',
       'barrio_Chacarita', 'barrio_Coghlan', 'barrio_Colegiales',
       'barrio_Congreso', 'barrio_Constitución', 'barrio_Flores',
       'barrio_Floresta', 'barrio_Las Cañitas', 'barrio_Liniers',
       'barrio_Mataderos', 'barrio_Monserrat', 'barrio_Monte Castro',
       'barrio_Nuñez', 'barrio_Once', 'barrio_Palermo',
       'barrio_Parque Avellaneda', 'barrio_Parque Centenario',
       'barrio_Parque Chacabuco', 'barrio_Parque Chas',
 